In [ ]:
!git clone "https://github.com/priyanshu11137/cs637.git" 

In [1]:
# Please note gait and drift datasets need to be loaded first into kaggle/collab we used kaggle for this(since the gait dataset is small it can be uploaded easily)
# this file is aimed at reproducing results of gait and drift dataset(drift dataset can be used directly in colab using google drive link "https://drive.google.com/drive/folders/12ssAwZ8BhOiXMypEFq6FHsx8Ui4nPT0Z?usp=drive_link")
# Secondly the approach developed(e value method by first defining threshold to minimize fdr) can also be run from here 

In [ ]:
%cd "/kaggle/working"

In [ ]:
!ls
%cd "cs637"
!ls
%cd 'ours'

In [ ]:
!pip install -r requirements.txt

In [ ]:
%cd 'gait'

In [ ]:
!pip install gdown
import gdown
# https://drive.google.com/file/d/1yAow6FhfvfaBsUUSszJS7f-HdCPQXakx/view?usp=drive_link
# https://drive.google.com/file/d/1V3rjEDaUTwc-KWROLkO9Q2ycVEyXuRdZ/view?usp=drive_link
# https://drive.google.com/file/d/1qMeptQPUOGnLiilsU2jSrIDmxyA0jx3K/view?usp=drive_link
# File 1
file_id_1 = "1V3rjEDaUTwc-KWROLkO9Q2ycVEyXuRdZ"
url_1 = f"https://drive.google.com/uc?id={file_id_1}"
pre_model = "file_1_name"  # Replace with desired file name
file_id_2 = "1qMeptQPUOGnLiilsU2jSrIDmxyA0jx3K"
url_2 = f"https://drive.google.com/uc?id={file_id_1}"
our_model = "file_2_name"  # Replace with desired file name
# https://drive.google.com/file/d/1uczjf4jjmfd7hgqqGeqAg6EWu8XydO3S/view?usp=drive_link
file_id_3 = "1uczjf4jjmfd7hgqqGeqAg6EWu8XydO3S"
url_3 = f"https://drive.google.com/uc?id={file_id_1}"
drift_model = "file_2_name"  # Replace with desired file name
gdown.download(url_1, pre_model, quiet=False)
gdown.download(url_2, our_model, quiet=False)



In [ ]:
import gdown
import zipfile
import os

# File ID and download URL
file_id_2 = "13F5yuIF6a2GKBXT55AHoZBGdcpo4MJDc"
url_2 = f"https://drive.google.com/uc?id={file_id_2}"

# Specify the file paths
download_path = "gait_logs.zip"  # Temporary file for the downloaded zip
extract_path = "/kaggle/working/gait_logs"  # Directory to extract the files

# Download the file
gdown.download(url_2, download_path, quiet=False)

# Unzip the file
os.makedirs(extract_path, exist_ok=True)
with zipfile.ZipFile(download_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print(f"Files extracted to: {extract_path}")

# Optionally, delete the zip file after extraction
os.remove(download_path)


In [ ]:
#use pre made gait logs to run files on new code
!python check_OOD_gait_new.py --save_dir '/kaggle/working/gait_logs' --ckpt {pre_model}  --transformation_list high_pass low_high high_low identity --wl 18 --cuda --gpu 0 --n 100 --disease_type all  --cal_root_dir '/kaggle/input/gait-dataset/gait-in-neurodegenerative-disease-database-1.0.0' --in_test_root_dir '/kaggle/input/gait-dataset/gait-in-neurodegenerative-disease-database-1.0.0' --out_test_root_dir '/kaggle/input/gait-dataset/gait-in-neurodegenerative-disease-database-1.0.0' --check_ood True

In [ ]:
#zipping file to download
import os
import zipfile

def zip_folder(folder_path, output_zip_file):
    """
    Zips the contents of a folder.
    
    Args:
        folder_path (str): Path to the folder to be zipped.
        output_zip_file (str): Name of the output zip file.
    """
    with zipfile.ZipFile(output_zip_file, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                file_path = os.path.join(root, file)
                arcname = os.path.relpath(file_path, folder_path)
                zipf.write(file_path, arcname)
    print(f"Folder '{folder_path}' has been zipped into '{output_zip_file}'.")

# Example Usage
folder_to_zip = '/kaggle/working/cs637/ours/gait/gait_log'  # Change this to your folder path
output_zip_name = '/kaggle/working/cs637/ours/gait/gait_log.zip'  # Output zip file name
zip_folder(folder_to_zip, output_zip_name)


In [ ]:
#run pre-trained-model and new model by changing model
!python check_OOD_gait.py --save_dir gait_log/ --ckpt {pre_model}  --transformation_list high_pass low_high high_low identity --wl 18 --cuda --gpu 0 --n 100 --disease_type all  --cal_root_dir '/kaggle/input/gait-dataset/gait-in-neurodegenerative-disease-database-1.0.0' --in_test_root_dir '/kaggle/input/gait-dataset/gait-in-neurodegenerative-disease-database-1.0.0' --out_test_root_dir '/kaggle/input/gait-dataset/gait-in-neurodegenerative-disease-database-1.0.0'

In [ ]:
#change w to change window length the below command is to start training model, logs are saved in saved_models
!python train_gait.py --log saved_models --transformation_list high_pass low_high high_low identity --wl 16 --cal_root_dir '/kaggle/input/gait-dataset/gait-in-neurodegenerative-disease-database-1.0.0' --train_root_dir '/kaggle/input/gait-dataset/gait-in-neurodegenerative-disease-database-1.0.0' --test_root_dir '/kaggle/input/gait-dataset/gait-in-neurodegenerative-disease-database-1.0.0'

In [ ]:
#without moving to gait folder in ours folder check results for drift dataset
!python check_OOD_drift.py --save_dir gait_log/ --ckpt '/kaggle/working/cs637/ours/gait/saved_models/best_drift_model.pt'  --transformation_list high_pass low_high high_low identity --wl 16 --cuda --gpu 0 --n 100 --disease_type all  --cal_root_dir '/kaggle/input/gait-dataset/gait-in-neurodegenerative-disease-database-1.0.0' --in_test_root_dir '/kaggle/input/gait-dataset/gait-in-neurodegenerative-disease-database-1.0.0' --out_test_root_dir '/kaggle/input/gait-dataset/gait-in-neurodegenerative-disease-database-1.0.0'